In [1]:
# 코드 8-26 라이브러리 호출
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb
import tensorflow as tf
import numpy as np

2024-12-03 02:06:02.167348: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1733159162.177444  256779 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1733159162.180215  256779 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-03 02:06:02.192800: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# 코드 8-27 하이퍼파라미터 초기화
max_features = 5000
maxlen = 400
batch_size = 32
embedding_dims = 50
filters = 250
kernel_size = 3
hidden_dims = 250
epochs = 300
np.random.seed(7)

In [3]:
# 코드 8-28 체크포인트 설정
import os

checkpoint_path = "training_1/cp.keras"
checkpoint_dir = os.path.dirname(checkpoint_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, verbose=1)

In [4]:
# 코드 8-29 데이터셋 메모리로 로딩
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
# 모양(길이)이 같은 벡터를 만들기 위해 ‘0’으로 시퀀스를 채웁니다.
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)

In [5]:
# 코드 8-30 모델 생성
model = Sequential()
model.add(Embedding(max_features, embedding_dims, input_length=maxlen))
# 무작위로 20%의 뉴런을 다음 층의 입력에서 무시
model.add(Dropout(0.2))
model.add(Conv1D(filters, kernel_size, padding="valid", activation="relu", strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation("relu"))
model.add(Dense(1))
model.add(Activation("sigmoid"))
model.summary()

/home/halozhan/Study/.venv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
I0000 00:00:1733159198.567572  256779 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6687 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:0a:00.0, compute capability: 6.1


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_max_pooling1d            │ ?                      │             0 │
│ (GlobalMaxPooling1D)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ ?                      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [7]:
# 코드 8-31 조기 종료가 추가되지 않은 모델 훈련
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=[cp_callback],
)

Epoch 1/300


I0000 00:00:1732668690.202821 1526501 service.cc:148] XLA service 0x7f73780015c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1732668690.205321 1526501 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce GTX 1080, Compute Capability 6.1
2024-11-27 09:51:30.439274: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1732668690.896104 1526501 cuda_dnn.cc:529] Loaded cuDNN version 90300


 32/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.5089 - loss: 0.6954

I0000 00:00:1732668694.211701 1526501 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


624/625 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6842 - loss: 0.5499
Epoch 1: saving model to training_1/cp.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - accuracy: 0.6845 - loss: 0.5496 - val_accuracy: 0.8764 - val_loss: 0.2980
Epoch 2/300
619/625 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9026 - loss: 0.2449
Epoch 2: saving model to training_1/cp.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.9026 - loss: 0.2448 - val_accuracy: 0.8946 - val_loss: 0.2571
Epoch 3/300
617/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9413 - loss: 0.1590
Epoch 3: saving model to training_1/cp.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9412 - loss: 0.1591 - val_accuracy: 0.8908 - val_loss: 0.2873
Epoch 4/300
616/625 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9663 - loss: 0.0958
Epoch 4: saving model to training_1/cp.keras
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9662 - loss: 0.0960 - val_accuracy: 0.8860 - val_loss: 0.3173
Epoch 5/300
61

In [10]:
# 코드 8-32 모델 평가
from tensorflow.keras.models import load_model

model = load_model(checkpoint_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print(
    f"Score: {model.metrics_names[0]} of {scores[0]}; {model.metrics_names[1]} of \
{scores[1]*100}%"
)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8757 - loss: 1.9472
Score: loss of 1.908522605895996; compile_metrics of 87.52400279045105%


In [11]:
# 코드 8-33 조기 종료를 적용한 모델 훈련

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

keras_callbacks = [
    EarlyStopping(monitor="val_loss", patience=30, mode="min", min_delta=0.0001),
    ModelCheckpoint(
        checkpoint_path, monitor="val_loss", save_best_only=True, mode="min"
    ),
]

model.fit(
    x_train,
    y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
    callbacks=keras_callbacks,
)

Epoch 1/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 6ms/step - accuracy: 0.9999 - loss: 5.6998e-04 - val_accuracy: 0.8782 - val_loss: 1.7120
Epoch 2/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9991 - loss: 0.0041 - val_accuracy: 0.8886 - val_loss: 1.7274
Epoch 3/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.9992 - loss: 0.0022 - val_accuracy: 0.8864 - val_loss: 1.4311
Epoch 4/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9997 - loss: 0.0011 - val_accuracy: 0.8872 - val_loss: 1.5821
Epoch 5/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9998 - loss: 4.2109e-04 - val_accuracy: 0.8852 - val_loss: 1.4359
Epoch 6/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 1.0000 - loss: 1.3832e-04 - val_accuracy: 0.8840 - val_loss: 1.8859
Epoch 7/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 5ms/step - accuracy: 0.9996 - loss: 0.0019 - val_accuracy: 0.8860 - val_loss: 1.7044
Epoch 8/300
625/625 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 1.0000 - loss: 1.713

In [ ]:
# 코드 8-34 모델 평가
model = load_model(checkpoint_path)
scores = model.evaluate(x_test, y_test, verbose=1)
print(
    f"Score: {model.metrics_names[0]} of {scores[0]}; \n\
{model.metrics_names[1]} of {scores[1]*100}%"
)

782/782 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8776 - loss: 1.4821
Score: loss of 1.4456533193588257; 
compile_metrics of 87.56399750709534%
